At this point, I've figure out how to perform docking in Python using Vina, and now want to implement it into functions so that it works nicely with the current SECS pipeline. 


In [11]:
import numpy as np
from vina import Vina
import subprocess

In [12]:
def prepare_ligand(
    lig_name,
    lig_smiles,
    prepare_ligand_script_path
):
    subprocess.run(
    ["sh", prepare_ligand_script_path, lig_name, lig_smiles],
    check=True
    );

def prepare_receptor(
    receptor_name,
    receptor_pdb_path,
    prepare_receptor_script_path
):
    subprocess.run(
    ["sh", prepare_receptor_script_path, receptor_name, receptor_pdb_path],
    check=True
    );

In [13]:
lig_name = "PF74"
lig_smiles = "CN(C([C@H](CC1=CC=CC=C1)NC(CC2=C(C)NC3=C2C=CC=C3)=O)=O)C4=CC=CC=C4"
prepare_ligand_script_path = "script/prepare_ligand.sh"
prepare_ligand(
    lig_name,
    lig_smiles,
    prepare_ligand_script_path
)

Scrub completed.
Summary of what happened:
Input molecules supplied: 1
mols processed: 1, skipped by rdkit: 0, failed: 0
nr isomers (tautomers and acid/base conjugates): 1 (avg. 1.000 per mol)
nr conformers:  1 (avg. 1.000 per isomer, 1.000 per mol)


/opt/anaconda3/envs/molpert/lib/python3.10/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0
No duplicate molecule molecule names were found


/opt/anaconda3/envs/molpert/lib/python3.10/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0


In [14]:
receptor_name = "4XFZ"
receptor_pdb_path = "reference/4xfz_two_chain_dry_clean.pdb"
prepare_receptor_script_path = "script/prepare_receptor.sh"

prepare_receptor(
    receptor_name,
    receptor_pdb_path,
    prepare_receptor_script_path
)

/opt/anaconda3/envs/molpert/lib/python3.10/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
@> 4919 atoms and 1 coordinate set(s) were parsed in 0.02s.



Files written:
4XFZ.pdbqt <-- static (i.e., rigid) receptor input file


In [20]:
def vina_setup_receptor(
    receptor_pdbqt,
    grid_center,
):
# https://github.com/ccsb-scripps/AutoDock-Vina/tree/develop
# https://github.com/janash/iqb-2024/blob/main/docking_single_ligand.ipynb
# https://autodock-vina.readthedocs.io/en/latest/docking_python.html
    v = Vina(sf_name='vina')
    v.set_receptor(rigid_pdbqt_filename=receptor_pdbqt)
    v.compute_vina_maps(center=grid_center, box_size=[20, 20, 20])
    return v 

def vina_dock_ligand(
    v, # vina model
    ligand_pdbqt,
    output_folder,
    output_name,
    exhaustiveness=100
):
# https://github.com/ccsb-scripps/AutoDock-Vina/tree/develop
# https://github.com/janash/iqb-2024/blob/main/docking_single_ligand.ipynb
# https://autodock-vina.readthedocs.io/en/latest/docking_python.html
    v.set_ligand_from_file(ligand_pdbqt)    
    # Dock the ligand
    v.dock(exhaustiveness=exhaustiveness)
    v.optimize()
    v.write_poses(output_folder+output_name, n_poses=5, overwrite=True)
    return v

In [21]:
receptor_pdbqt = "./4XFZ.pdbqt"
grid_center    = [19.61, -23.07, 0.3]
ligand_pdbqt   = "prepared_ligands/PF74-1.pdbqt"
output_folder  = "docked_poses/"
output_name    = "4XFZ_PF74_test.pdbqt"

v = vina_dock_ligand(
        vina_setup_receptor(receptor_pdbqt,grid_center),
        ligand_pdbqt,
        output_folder,
        output_name
)

Computing Vina grid ... done.
Performing docking (random seed: 934508013) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -9.614          0          0
   2       -8.996      1.304      2.044
   3        -8.92      1.958      7.899
   4       -8.595      2.415       7.46
   5       -8.532      5.038      8.381
   6       -8.505     0.9995      2.396
   7       -8.503      1.165      1.755
   8       -8.397       3.16      6.662
   9       -8.332      2.334      4.421
  10       -8.289       1.45      4.385
  11       -8.107       1.82      3.583
  12       -8.105      1.706      2.775
  13       -7.994      4.132      7.883
  14       -7.969      4.566      7.657
  15       -7.958      3.836      6.894
  16       -7.863      1.518      3.866

In [22]:
v.energies()[0,0]

-9.614